In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from pandarallel import pandarallel

In [8]:
df = pd.read_csv('avazu.csv')

In [9]:
df.drop(columns=['id'],inplace=True)

In [10]:
df.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,14102100,1005,0,9e8cf15d,0d3cb7be,f028772b,ecad2386,7801e8d9,07d7df22,...,1,2,20596,320,50,2161,0,35,100148,157
1,0,14102100,1005,1,0c2fe9d6,27e3c518,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,6558,320,50,571,2,39,-1,32
2,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,15706,320,50,1722,0,35,-1,79
3,1,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,54c5d545,2347f47a,0f2161f8,...,1,0,21611,320,50,2480,3,297,100111,61
4,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,15706,320,50,1722,0,35,-1,79


In [11]:
sparse_features = [f for f in df.columns]
sparse_features

['click',
 'hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21']

In [7]:
pandarallel.initialize()
def replace_values(series):
    counts = series.value_counts()
    return series.apply(lambda x: -1 if counts[x] < 4 else x)
df = df.parallel_apply(replace_values)

INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [12]:
df = df.astype(str)
tk0 = tqdm(sparse_features, desc='LabelEncoder')
for feat in tk0:
    lbe = LabelEncoder()
    df[feat] = lbe.fit_transform(df[feat])
print([df[f].max()+1 for f in sparse_features])
df.to_csv('preprocessed_avazu.csv', index=False)

LabelEncoder: 100%|██████████| 23/23 [04:27<00:00, 11.64s/it]


[2, 240, 7, 7, 4737, 7745, 26, 8552, 559, 36, 2686408, 6729486, 8251, 5, 4, 2626, 8, 9, 435, 4, 68, 172, 60]
